# Figure 1
___

In [13]:
library(reshape2)
library(dplyr)

library(ggplot2)
# library(cowplot)

In [7]:
parent <- '/prj/Florian_Leuschner_spatial/analysis/Nanopore/' # change this path!
config <- yaml::yaml.load_file(file.path(parent, 'ScNaST', 'workflow', 'config.yaml', fsep=.Platform$file.sep))

local <- getwd() # you might have to adjust this path to make the local files visible

Nanopore saturation
---

We used [RSeQC](http://rseqc.sourceforge.net/) with default parameters and `-r gffcmp.multi_exons.annotated.bed`. The gene model to annotate splicing junctions is converted to BED format from the **ScNaST** *de novo* annotation.


In [10]:
data <- read.table(file.path(local, 'saturation.txt', fsep=.Platform$file.sep))
percent_reads <- data[1,2:ncol(data)]
data$V1[grep('known', data$V1)] <- 'Known junctions'
data$V1[grep('all', data$V1)] <- 'All junctions'
data <- data[grep('novel', data$V1, invert=T),]
data$V1 <- factor(data$V1, levels=c('All junctions', 'Known junctions'))

f1 <- data %>%
    `[`(-1,) %>%
    melt(id.vars='V1') %>%
    mutate(percent_reads=as.numeric(rep(percent_reads, each=8))) %>%
    mutate(value=value/1000) %>%
        group_by(V1, percent_reads) %>%
      summarise( 
        n=n(),
        mean=mean(value),
        sd=sd(value)
      ) %>%
      mutate( se=sd/sqrt(n))  %>%
      mutate( ic=se * qt((1-0.05)/2 + .5, n-1)) %>%
        ggplot(aes(x=percent_reads, y=mean, color=V1)) + 
            geom_errorbar(aes(ymin=mean-se, ymax=mean+se), width=.1) +
            geom_line(size=1.25) + 
            scale_color_grey(start=.75, end=0) +
                labs(x="Percent of total reads", y="Spliced junctions (x1000)") + 
                theme_minimal() +
                theme(legend.title = element_blank(),
                      legend.text=element_text(size=18),
                      legend.position = "top",
                      axis.text = element_text(size = 18),
                      axis.title.x = element_text(size = 18),
                      axis.title.y = element_text(size = 18),
                      panel.grid.minor.x = element_blank(),
                      panel.grid.minor.y = element_blank())

`summarise()` has grouped output by 'V1'. You can override using the `.groups` argument.


Nanopore/Illumina read coverage
---

We used [RSeQC](http://rseqc.sourceforge.net/) with default parameters. For Nanopore, we used `-r gffcmp.multi_exons.annotated.bed`, as above. For illumina, we used the default CellRanger annotations (`mm10-2020-A_build` converted to BED format).

In [11]:
data <- read.table(file.path(local, 'coverage.txt', fsep=.Platform$file.sep))
data <- data[2:nrow(data),1:ncol(data)]
samples <- data[,1]
samples[grep('Nanopore', samples)] <- 'Nanopore'
samples[grep('Illumina', samples)] <- 'Illumina'

f2 <- t(apply(data[,2:ncol(data)], 1, function(x) {
    (x - min(x))/(max(x)-min(x))
} )) %>% data.frame() %>%
    mutate(sample=samples) %>%
    melt(id.vars='sample') %>%
    mutate(variable = rep(1:100, each=length(samples))) %>%
        group_by(sample, variable) %>%
      summarise( 
        n=n(),
        mean=mean(value),
        sd=sd(value)
      ) %>%
      mutate( se=sd/sqrt(n))  %>%
      mutate( ic=se * qt((1-0.05)/2 + .5, n-1)) %>%
        ggplot(aes(x=variable, y=mean, color=sample)) + 
                geom_errorbar(aes(ymin=mean-se, ymax=mean+se), width=.1) +
                geom_line(size=1.25) +
                scale_color_grey(start=.75, end=0) +
                labs(x="Transcript percentile (from 5' to 3')", y="Normalized coverage") + 
                theme_minimal() +
                theme(legend.title = element_blank(),
                      legend.text=element_text(size=18),
                      legend.position = "top",
                      axis.text = element_text(size = 18),
                      axis.title.x = element_text(size = 18),
                      axis.title.y = element_text(size = 18),
                      panel.grid.minor.x = element_blank(),
                      panel.grid.minor.y = element_blank())

`summarise()` has grouped output by 'sample'. You can override using the `.groups` argument.


Summary of Nanopore reads identified by ScNapBar at each processing step
---


In [12]:
# prepare data for plotting
df <- purrr::map2(config$samples, names(config$samples), function(.x, .y) {
    log <- file.path(dirname(.x), 'real.log', fsep=.Platform$file.sep)
    label <- file.path(dirname(.x), 'real.label', fsep=.Platform$file.sep)
    # total reads
    data <- dirname(dirname(.x))
    name <- paste(unlist(strsplit(basename(.x), '.', fixed=T))[1], 'fastq.gz', sep='.')
    total <- system(paste0("awk '{s++}END{print s/4}' ", file.path(data, 'data', name, fsep=.Platform$file.sep)), intern=TRUE)
    # assigned barcodes
    assigned <- system(paste0('wc -l ', label), intern=TRUE)
    # scNapBar log
    x <- read.table(log, sep='\t')
    x <- rbind(c('Total reads', total), x)
    x <- rbind(x, c('Assigned to barcode', unlist(strsplit(assigned, ' '))[1]))
    x[,2] <- as.integer(x[,2])
    x <- mutate(x, Assigned=V2/lag(V2)*100)
    # x <- x[-1,]
    # add sample name
    x$Sample <- toupper(substr(.y, nchar(.y), nchar(.y)))
    x
})
df <- do.call("rbind", df)
df$Sample <-  factor(df$Sample, levels=rev(c('A', 'B', 'C', 'D')))
df$Steps <- factor(df[,1], levels=rev(c('Total reads', 'Aligned to genome', 'Aligned to adapter', 'Aligned to barcode', 'Assigned to barcode')))

x <- df[,c(1,2,4,5)]
x <- x %>%
  group_by(Steps) %>%
  summarise( 
    n=n(),
    mean=mean(V2),
    sd=sd(V2)
  ) %>%
  mutate( se=sd/sqrt(n))  %>%
  mutate( ic=se * qt((1-0.05)/2 + .5, n-1))
f3 <- ggplot(x) +
  geom_bar(aes(x=Steps, y=mean, fill=Steps), stat="identity", color="black", alpha=0.5) +
  geom_errorbar(aes(x=Steps, ymin=mean-se, ymax=mean+se), width=0.1, colour="black", alpha=0.9, size=1) +
  scale_fill_grey(start=1, end=0) +
  coord_flip() + 
  labs(x="", y="Reads") + 
    theme_minimal() +
    theme(legend.position = 'None',
          panel.grid.major.y = element_blank(),
          axis.text = element_text(size = 18),
          axis.title.x = element_text(size = 18),
          axis.title.y = element_blank())

In [34]:
# combine 3 figures into bottom panel
p <- ggpubr::ggarrange(f1, f2, f3, 
          labels = c("B", "C", "D"),
          ncol = 3, nrow = 1, font.label = list(size = 20))
filen <- file.path(parent, 'ScNaST', 'paper', 'figures', '1', fsep=.Platform$file.sep)
filen <- file.path(filen, 'panel2.pdf')
ggsave(filen, width=24, height=8, dpi=1200)


In [35]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 10 (buster)

Matrix products: default
BLAS:   /beegfs/biosw/R/4.1.1_deb10/lib/R/lib/libRblas.so
LAPACK: /beegfs/biosw/R/4.1.1_deb10/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] cowplot_1.1.1  reshape2_1.4.4 dplyr_1.0.7    ggplot2_3.3.5 

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-6     tidyselect_1.1.1 repr_1.1.3       purrr_0.3.4     
 [5] carData_3.0-4    colorspace_2.0-2 vctrs_0.3.8      generics_0.1.1  
 [